In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
import io
from sklearn import svm, base, feature_selection, linear_model
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve,auc,classification_report,f1_score,accuracy_score,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import subprocess
from os import listdir
#to visualize
from sklearn.externals.six import StringIO
# import pydot

D:\Users\baran\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
# Read data files
df = pd.read_csv('features_final.csv').drop(columns='Unnamed: 0')
model_test = pd.read_csv('ENZYMES_g300_final.csv').drop(columns='Unnamed: 0')

In [3]:
# Prepare data to train. 
# X: Independent values
# y: Target values (Categories)
def prepareData(df, normalize=False):
    X = np.array(df.drop(['Name', 'Categories'], 1))
    if normalize:
        return (X - X.mean()) / (X.max() - X.min()), np.array(df['Categories'])
    return X, np.array(df['Categories'])

In [43]:
def doSVMcvPrediction(Xin,yin,Xtest,ytest,modType):
    #input: training/test data and labels, model type
    #supported models: SVM (l1 and l2), AdaBoost, decision tree, and random forest
    #train with 5-fold cross validation, then test once using test (holdout) data
    #once the best estimator is chosen here, train on the entire dataset (in + test) outside this function
    #output: training accuracy, gereralization accuracy, feature weights/importances, classifier, 
    #  classification report, training f1-score and generalization f1-score
    nfolds = 5
    cv = StratifiedKFold(n_splits=nfolds, shuffle=True)
    #l1 penalty enforces sparsity in weights, only available for linear SVM classifier
    if modType in ('SVM-L2','svm-l2'):
        clasf = svm.LinearSVC(loss='squared_hinge', penalty='l2', tol=.001, dual=False, class_weight='balanced')
        cvclasf = GridSearchCV(clasf, param_grid = {
            'C' : [0.05, 0.1, 0.5, 1, 5, 10, 500, 1000]
            }, verbose=0,refit=True,
            cv=cv,
            # scoring='roc_auc',
            scoring='f1_weighted',
        n_jobs=4)

    elif modType in ('SVM-L1','svm-l1'):
        clasf = svm.LinearSVC(loss='squared_hinge', penalty='l1', tol=.001, dual=False, class_weight='balanced')
        cvclasf = GridSearchCV(clasf, param_grid = {
            'C' : [0.05, 0.1, 0.5, 1, 5, 10, 500, 1000]
            }, verbose=0,refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)
    
    #decision tree classifiers
    elif modType in ('ada','adaboost','adaboost-tree'):
        clasf = AdaBoostClassifier()
        cvclasf = GridSearchCV(clasf, param_grid = {
            'n_estimators' : [5,10,25,50,100],
            'learning_rate' : [0.1,0.3,0.5]
            }, verbose=0,refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)

    elif modType in ('dtree','decision-tree'):
        clasf = DecisionTreeClassifier()
        cvclasf = GridSearchCV(clasf, param_grid = {
            'splitter' : ['best'],
            'criterion' : ['entropy','gini'],
            'max_features' : [0.2,'sqrt',1.],
            'max_depth' : [2,4], 
            'class_weight' : ['balanced'], 
            }, verbose=0,refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)

    elif modType in ('rf','random-forest'):
        clasf = RandomForestClassifier()
        cvclasf = GridSearchCV(clasf, param_grid = {
            'n_estimators' : [5,10,25,50,100],
            'criterion' : ['entropy','gini'],
            'max_features' : [0.2,'sqrt',1.],
            'max_depth' : [2,4], 
            'class_weight' : ['balanced'], 
            }, verbose=0,refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)
    
    elif modType in ('knn', 'kneighbors'):
        clasf = KNeighborsClassifier()
        cvclasf = GridSearchCV(clasf, param_grid = {
            'n_neighbors': [3, 5, 7, 11, 13, 15, 19],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
            }, verbose=0, refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)

    cvclasf.fit(Xin,yin)
    bclasf = cvclasf.best_estimator_
    print("%s %d-fold CV params: %s" % (modType,nfolds,cvclasf.best_params_))
    
    if modType in ('ada','adaboost-tree','dtree','decision-tree','rf','random-forest'):
        w = bclasf.feature_importances_
    elif modType in ('SVM-L1','svm-l1','SVM-L2','svm-l2'):
        w = bclasf.coef_
    elif modType in ('knn', 'kneighbors'):
        w = np.zeros(2)
    
    bclasf.fit(Xin,yin)
    y_train_pred = bclasf.predict(Xin)
    acTrain = accuracy_score(yin,y_train_pred)
    f1Train = f1_score(yin,y_train_pred,average="weighted")
    
    y_pred = bclasf.predict(Xtest)
    report = classification_report(ytest, y_pred)
    acGeneral = accuracy_score(ytest, y_pred)
    f1Gen = f1_score(ytest,y_pred,average="weighted")

    return(acTrain,np.squeeze(w),bclasf,report,(acTrain,acGeneral),(f1Train,f1Gen))

In [46]:
def testModel(df_train, df_testing, modelType):
    X, y = prepareData(df_train)
    X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.5, stratify=y)
    
    RGfamilies = ["CL", "CNFG", "GNP", "PA"]
    
    # Preprocess
    scaler1=StandardScaler()
    X_train = scaler1.fit_transform(X_train.astype(np.double))
    X_test = scaler1.transform(X_test.astype(np.double))
    
    score,optWeights,clasf,rep,accs,f1s = doSVMcvPrediction(X_train, y_train, X_test, y_test, modelType)
    
    print(rep)
    print()
    
    test_measure = df_testing.drop(columns=['Name'])
    x = np.array(test_measure.values)
    
    scaler=StandardScaler()
    X = scaler.fit_transform(X.astype(np.double))
    x = scaler.transform(x.astype(np.double))
    
    clasf.fit(X,y)
    novelRG = clasf.predict(x)
    print("\nPrediction is %s\n"%(novelRG))
    
    Fcol = df.drop(['Name', 'Categories'], 1).columns
    
    if modelType in ('SVM-L1','SVM-L2'):
        #TODO: print statements when useSpectral
        print("SVM feature weights:")
        print(np.column_stack((clasf.coef_.T,Fcol)))
        sc = x.dot(clasf.coef_.T) + clasf.intercept_
        print("scores:")
        print(np.vstack((RGfamilies,sc)))
    elif modelType in ('adaboost-tree'):
        print("Adaboost feature weights:")
        print(np.column_stack((clasf.feature_importances_.T,Fcol)))
        # print "prediction probabilities:"
        # print np.vstack((RGfamilies,clasf.predict_proba(x)))
        print("decision function:")
        print(np.vstack((RGfamilies,clasf.decision_function(x))))
        #also the actual tree?
    elif modelType in ('decision-tree'):
        print("Decision tree feature weights:")
        print(np.column_stack((clasf.feature_importances_.T,Fcol)))
        print("prediction probabilities:")
        print(np.vstack((RGfamilies,clasf.predict_proba(x))))
        #also the actual tree
        #Fcol2 = [s.replace('n3','H').replace('n4','F') for s in Fcol]
        #writeTree(clasf,Fcol2,graphFolder[:-1]+'_dTree.pdf')
    elif modelType in ('random-forest'):
        print("Random Forest feature weights:")
        print(np.column_stack((clasf.feature_importances_.T,Fcol)))
        print("prediction probabilities:")
        print(np.vstack((RGfamilies,clasf.predict_proba(x))))
    elif modelType in ('knn'):
        print("K Nearest Neighbors feature weights:")
        #print(np.column_stack((clasf.feature_importances_.T,Fcol)))
        print("prediction probabilities:")
        print(np.vstack((RGfamilies,clasf.predict_proba(x))))

In [24]:
testModel(df, model_test, "SVM-L1")

SVM-L1 5-fold CV params: {'C': 1}
              precision    recall  f1-score   support

          CL       0.98      1.00      0.99        50
        CNFG       0.85      0.92      0.88        50
         GNP       0.87      0.82      0.85        50
          PA       1.00      0.96      0.98        50

    accuracy                           0.93       200
   macro avg       0.93      0.92      0.92       200
weighted avg       0.93      0.93      0.92       200



Prediction is ['GNP']

SVM feature weights:
[[-0.9156082305847411 0.0 0.1978956491516132 0.0 'Vertex']
 [0.0 1.1115347204447956 0.0 0.0 'Edges']
 [0.0 0.0 0.0 0.0 'Density']
 [0.034664468900289 0.7433238836780317 0.0 0.26881327927819937
  'Max. Degree']
 [0.0 0.17164728152363842 0.0 0.0 'Avg. Degree']
 [0.0 0.003859230665027998 0.049781172285455415 -0.799430199569762
  'Max. k-core']
 [0.0 0.0 -0.5587277792408188 0.2282874217404786 'Avg. Clustering Coeff.']
 [0.04985843604343833 0.061769115371406495 -0.24720435527244194
  -

D:\Users\baran\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Users\baran\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Users\baran\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [25]:
testModel(df, model_test, "SVM-L2")

SVM-L2 5-fold CV params: {'C': 5}
              precision    recall  f1-score   support

          CL       0.98      1.00      0.99        50
        CNFG       0.90      0.88      0.89        50
         GNP       0.88      0.88      0.88        50
          PA       1.00      1.00      1.00        50

    accuracy                           0.94       200
   macro avg       0.94      0.94      0.94       200
weighted avg       0.94      0.94      0.94       200



Prediction is ['GNP']

SVM feature weights:
[[-0.869708431232342 -0.10831842317664232 0.5135925436600742
  0.08332674526942066 'Vertex']
 [-0.4659275721352415 1.8537902035907525 0.6619584852135462
  -0.2441087568530374 'Edges']
 [-0.2931538430069096 1.736548069137694 0.4588865561063264
  -0.23724017832995756 'Density']
 [0.1991621181774044 0.6912121700778762 0.2999407923439558
  0.18948078961072506 'Max. Degree']
 [-0.37920216006514246 1.7997636296641972 0.5597467240443557
  -0.24136612080531875 'Avg. Degree']
 [-0.04742512

In [26]:
testModel(df, model_test, "adaboost-tree")

adaboost-tree 5-fold CV params: {'learning_rate': 0.1, 'n_estimators': 5}
              precision    recall  f1-score   support

          CL       0.98      1.00      0.99        50
        CNFG       0.76      0.90      0.83        50
         GNP       0.87      0.68      0.76        50
          PA       0.98      1.00      0.99        50

    accuracy                           0.90       200
   macro avg       0.90      0.90      0.89       200
weighted avg       0.90      0.90      0.89       200



Prediction is ['PA']

Adaboost feature weights:
[[0.0 'Vertex']
 [0.0 'Edges']
 [0.0 'Density']
 [0.0 'Max. Degree']
 [0.0 'Avg. Degree']
 [0.4 'Max. k-core']
 [0.0 'Avg. Clustering Coeff.']
 [0.0 'Diameter']
 [0.0 'Average Path Length']
 [0.0 'Total Triangles']
 [0.2 'Avg. Eigenvector Centrality']
 [0.4 'f1']
 [0.0 'f2']
 [0.0 'f3']
 [0.0 'f4']
 [0.0 'f5']
 [0.0 'f6']
 [0.0 'f7']
 [0.0 'f8']
 [0.0 'f9']
 [0.0 'f10']
 [0.0 'f11']
 [0.0 'f12']
 [0.0 'f13']
 [0.0 'f14']
 [0.0 'f15']
 [0

In [27]:
testModel(df, model_test, "decision-tree")

decision-tree 5-fold CV params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'max_features': 1.0, 'splitter': 'best'}
              precision    recall  f1-score   support

          CL       0.98      0.98      0.98        50
        CNFG       0.89      0.98      0.93        50
         GNP       0.96      0.86      0.91        50
          PA       1.00      1.00      1.00        50

    accuracy                           0.95       200
   macro avg       0.96      0.95      0.95       200
weighted avg       0.96      0.95      0.95       200



Prediction is ['PA']

Decision tree feature weights:
[[0.35648809056158115 'Vertex']
 [0.0 'Edges']
 [0.0 'Density']
 [0.0 'Max. Degree']
 [0.0 'Avg. Degree']
 [0.3725151841376578 'Max. k-core']
 [0.0 'Avg. Clustering Coeff.']
 [0.0 'Diameter']
 [0.0 'Average Path Length']
 [0.0 'Total Triangles']
 [0.1637823306791414 'Avg. Eigenvector Centrality']
 [0.0 'f1']
 [0.0 'f2']
 [0.0 'f3']
 [0.0 'f4']
 [0.0 'f5']
 [0.0 'f6']
 

In [28]:
testModel(df, model_test, "random-forest")

random-forest 5-fold CV params: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 4, 'max_features': 1.0, 'n_estimators': 10}
              precision    recall  f1-score   support

          CL       0.98      1.00      0.99        50
        CNFG       0.82      0.80      0.81        50
         GNP       0.77      0.80      0.78        50
          PA       1.00      0.96      0.98        50

    accuracy                           0.89       200
   macro avg       0.89      0.89      0.89       200
weighted avg       0.89      0.89      0.89       200



Prediction is ['PA']

Random Forest feature weights:
[[0.13827904577197958 'Vertex']
 [0.04548528014205507 'Edges']
 [0.0 'Density']
 [0.0 'Max. Degree']
 [0.044521912045067925 'Avg. Degree']
 [0.1604337139968027 'Max. k-core']
 [0.0 'Avg. Clustering Coeff.']
 [0.0 'Diameter']
 [0.0052423190555837725 'Average Path Length']
 [0.0 'Total Triangles']
 [0.10598078303295295 'Avg. Eigenvector Centrality']
 [0.23705797398185

In [47]:
testModel(df, model_test, "knn")

knn 5-fold CV params: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
              precision    recall  f1-score   support

          CL       1.00      0.96      0.98        50
        CNFG       0.85      0.90      0.87        50
         GNP       0.86      0.84      0.85        50
          PA       1.00      1.00      1.00        50

    accuracy                           0.93       200
   macro avg       0.93      0.92      0.93       200
weighted avg       0.93      0.93      0.93       200



Prediction is ['GNP']

K Nearest Neighbors feature weights:
prediction probabilities:
[['CL' 'CNFG' 'GNP' 'PA']
 ['0.0' '0.0' '0.6724675368557238' '0.3275324631442762']]
